# Data acquisition:

In [21]:
import pandas as pd
from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup as bs

In [22]:
path = '../data/raw/raw_data_project_m1.db'
engine = create_engine(f'sqlite:///{path}')

In [23]:
# Table summary

pd.read_sql_query("SELECT * from sqlite_master WHERE type = 'table'", engine)

,type,name,tbl_name,rootpage,sql
0,table,personal_info,personal_info,2,CREATE TABLE personal_info (\n\tuuid TEXT NOT ...
1,table,country_info,country_info,401,CREATE TABLE country_info (\n\tuuid TEXT NOT N...
2,table,career_info,career_info,762,CREATE TABLE career_info (\n\tuuid TEXT NOT NU...
3,table,poll_info,poll_info,1168,CREATE TABLE poll_info (\n\tuuid TEXT NOT NULL...


In [24]:
# Table acquisition

df_personal_info = pd.read_sql_table('personal_info', engine)
df_country_info =  pd.read_sql_table('country_info', engine)
df_career_info =   pd.read_sql_table('career_info', engine)
df_poll_info =     pd.read_sql_table('poll_info', engine)

In [25]:
engine.dispose()

### Table 'personal_info'

In [26]:
print('personal_info table \n\n')
print(f'Rows: {df_personal_info.shape[0]}')
print(f'Columns: {df_personal_info.shape[1]}\n\n')

print('Null values:\n')
print(df_personal_info.isnull().sum(), '\n\n')

print('Data types:\n')
print(df_personal_info.dtypes)

df_personal_info.head(3)

df_personal_info.to_parquet('../data/raw/personal_info.parquet')

personal_info table 


Rows: 9649
Columns: 5


Null values:

uuid                0
age                 0
gender              0
dem_has_children    0
age_group           0
dtype: int64 


Data types:

uuid                object
age                 object
gender              object
dem_has_children    object
age_group           object
dtype: object


### Table 'country_info':

In [7]:
print('country_info table \n\n')
print(f'Rows: {df_country_info.shape[0]}')
print(f'Columns: {df_country_info.shape[1]}\n\n')

print('Null values:\n')
print(df_country_info.isnull().sum(), '\n\n')

print('Data types:\n')
print(df_country_info.dtypes)

df_country_info.head(3)

df_country_info.to_parquet('../data/raw/country_info.parquet')

country_info table 


Rows: 9649
Columns: 3


Null values:

uuid            0
country_code    0
rural           0
dtype: int64 


Data types:

uuid            object
country_code    object
rural           object
dtype: object


### Table 'career_info':

In [8]:
print('career_info table \n\n')
print(f'Rows: {df_career_info.shape[0]}')
print(f'Columns: {df_career_info.shape[1]}\n\n')

print('Null values:\n')
print(df_career_info.isnull().sum(), '\n\n')

print('Data types:\n')
print(df_career_info.dtypes)

df_career_info['normalized_job_code'].fillna('None', inplace=True) # Null to str

df_career_info.head(3)

df_career_info.to_parquet('../data/raw/career_info.parquet')

career_info table 


Rows: 9649
Columns: 4


Null values:

uuid                      0
dem_education_level     663
dem_full_time_job         0
normalized_job_code    3947
dtype: int64 


Data types:

uuid                   object
dem_education_level    object
dem_full_time_job      object
normalized_job_code    object
dtype: object


### Table 'poll_info':

In [9]:
print('poll_info table \n\n')
print(f'Rows: {df_poll_info.shape[0]}')
print(f'Columns: {df_poll_info.shape[1]}\n\n')

print('Null values:\n')
print(df_poll_info.isnull().sum(), '\n\n')

print('Data types:\n')
print(df_poll_info.dtypes)

df_poll_info.head(3)

df_poll_info.to_parquet('../data/raw/poll_info.parquet')

poll_info table 


Rows: 9649
Columns: 6


Null values:

uuid                                                   0
question_bbi_2016wave4_basicincome_awareness           0
question_bbi_2016wave4_basicincome_vote                0
question_bbi_2016wave4_basicincome_effect              0
question_bbi_2016wave4_basicincome_argumentsfor        0
question_bbi_2016wave4_basicincome_argumentsagainst    0
dtype: int64 


Data types:

uuid                                                   object
question_bbi_2016wave4_basicincome_awareness           object
question_bbi_2016wave4_basicincome_vote                object
question_bbi_2016wave4_basicincome_effect              object
question_bbi_2016wave4_basicincome_argumentsfor        object
question_bbi_2016wave4_basicincome_argumentsagainst    object
dtype: object


### API request df_career_info['normalized_job_code']

In [10]:
norm_job_codes = df_career_info['normalized_job_code'].unique().tolist()

norm_job_dic = {}

In [40]:
%%timeit
for i in norm_job_codes[1:]: #saltamos el none
    
    response = requests.get(f'http://api.dataatwork.org/v1/jobs/{i}')
    norm_job_dic[i] = (response.json())['title']

norm_job_dic['none'] = 'none' # Añadimos none al dic

1min 54s ± 1.95 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
pd.DataFrame(norm_job_dic.items(), columns = ['normalized_job_code', 'title']).to_parquet('../data/raw/norm_job_codes-names.parquet')

In [ ]:

from progress.bar import Bar

with Bar('Processing...') as bar:
     for i in norm_job_codes[1:]: #saltamos el none

            response = requests.get(f'http://api.dataatwork.org/v1/jobs/{i}')
            norm_job_dic[i] = (response.json())['title']
            bar.next()





In [37]:
def basic_widget_example():
    widgets = [progressbar.Percentage(), progressbar.Bar()]
    bar = progressbar.ProgressBar(widgets=widgets, max_value=10).start()
    for i in range(10):
        
        for i in norm_job_codes[1:]: #saltamos el none
            response = requests.get(f'http://api.dataatwork.org/v1/jobs/{i}')
            norm_job_dic[i] = (response.json())['title']
        
        bar.update(i + 1)
    bar.finish()

In [38]:
basic_widget_example()

TypeError: __init__() got an unexpected keyword argument 'max_value'

### API request country code

In [13]:
country_code_list = df_country_info['country_code'].unique().tolist()
country_dic = {}

In [14]:
for i in country_code_list:
    
    response = requests.get(f'https://restcountries.eu/rest/v2/alpha/{i}')
    country_dic[i] = (response.json())['name'] 

In [15]:
pd.DataFrame(country_dic.items(), columns=['country_code', 'country_name']).to_parquet('../data/raw/api_country_code-name.parquet')

### Web scraping country code

In [16]:
url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html = requests.get(url).content

In [17]:
soup = bs(html, 'lxml')
soup = soup.find_all('td')

lst = []

for i in soup:
    if str(i) != ' ':
        lst.append(str(i)[4:-6])

In [18]:
countries = []
codes = []

for i in range(0, len(lst[:56])):  # Hasta UK
    if i % 2 == 0:
        countries.append(lst[i])
    else:
        codes.append((lst[i].strip())[1:-1])

df_countries = pd.DataFrame(zip(countries, codes), columns=['country', 'code'])
df_countries.replace(' United Kingdom', 'United_Kingdom', inplace= True)
df_countries.to_parquet('../data/raw/webscraping_country_code-name.parquet')

In [20]:
df_countries

,country,code
0,Belgium,BE
1,Greece,EL
2,Lithuania,LT
3,Portugal,PT
4,Bulgaria,BG
5,Spain,ES
6,Luxembourg,LU
7,Romania,RO
8,Czechia,CZ
9,France,FR
